In [1]:
%load_ext autoreload
%autoreload 2
import tensorflow as tf

tf.config.list_physical_devices()

c:\ProgramData\Anaconda3\envs\thesis\lib\site-packages\h5py\__init__.py:36: UserWarning: h5py is running against HDF5 1.12.2 when it was built against 1.12.1, this may cause problems
  _warn(("h5py is running against HDF5 {0} when it was built against {1}, "


[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [2]:
import numpy as np
from loader.CSVLoader import csv_loader

gt,wave = csv_loader()


all_data = np.load("./experiment2/quan.npy")
test_data = np.load('./experiment2/quan_test.npy')

#all_data = np.ma.masked_array(all_data, mask=np.isnan(all_data)).filled(0)
#test_data = np.ma.masked_array(test_data, mask=np.isnan(test_data)).filled(0)

all_data.shape

(1732, 150, 11)

In [3]:
from creators.CNN3DCreator import  CNN3DNetworkParams
from creators.CRNNCreator import CRNNNetworkParams
from creators.ModelParams import Conv1DParams, Conv2DParams, Conv3DParams, DeepLayerParams, FlattenParams, GenericNeuralNetworkParams, RandomForestParams, RecurrentLayerParams, SVRParams
from creators.CNN1DCreator import  CNN1DNetworkParams
from creators.CNN2DCreator import  CNN2DNetworkParams
from prep.HelperBlocks import pipe, split_3d_to_timeseries
import tensorflow as tf
from prep.MSCBlock import msc_block, msc_block1d

from prep.Normlize import normalize_layers

drop=0.53
epochs=50
optimizer = tf.keras.optimizers.Adam
folds = 5


params_cnn3d = CNN3DNetworkParams(
    conv=[Conv3DParams(kernel=(10,10,1),pool_size=(10,10,1),filters=32,normalization=True,activation="relu", dropout=drop),
          Conv3DParams(kernel=(1,1,15),pool_size=(1,1,15),filters=128, normalization=True,activation="relu", dropout=drop),
          ],
    flatten=FlattenParams(dropout=drop,normalization=True),
    deep=[DeepLayerParams(neurons=1000,dropout=drop,normalization=True),
          DeepLayerParams(neurons=400,dropout=drop,normalization=True)],
    generic=GenericNeuralNetworkParams(
        prepare_function=lambda data, is_test: pipe(data,[
            lambda d: np.expand_dims(d, axis=-1),
            lambda d: normalize_layers(d,not is_test)
        ]),
        epochs=epochs,
        optimizer=optimizer
        )
)

params_cnn2d = CNN2DNetworkParams(
    conv=[Conv2DParams(kernel=(5,5),pool_size=(2,2),filters=64,normalization=True,activation="relu", dropout=drop),
          Conv2DParams(kernel=(3,3),pool_size=(2,2),filters=512, normalization=True,activation="relu", dropout=drop),
          Conv2DParams(kernel=(2,2),pool_size=(2,2),filters=512, normalization=True,activation="relu", dropout=drop),
          ],
    flatten=FlattenParams(dropout=drop,normalization=True),
    deep=[DeepLayerParams(neurons=1000,dropout=drop,normalization=True),
          DeepLayerParams(neurons=400,dropout=drop,normalization=True)],
    generic=GenericNeuralNetworkParams(
        prepare_function=lambda data, is_test: pipe(data,[
            lambda d: normalize_layers(d,not is_test)]), 
        epochs=epochs,
        optimizer=optimizer
        )
)

params_cnn1d = CNN1DNetworkParams(
    conv=[Conv1DParams(kernel=10,pool_size=5,filters=500,normalization=True,activation="relu", dropout=drop),
          Conv1DParams(kernel=5,pool_size=5,filters=1500, normalization=True,activation="relu", dropout=drop)
          ],
    deep=[DeepLayerParams(neurons=1000,dropout=drop,normalization=True),
          DeepLayerParams(neurons=400,dropout=drop,normalization=True)],
          flatten=FlattenParams(dropout=drop,normalization=True),
    generic=GenericNeuralNetworkParams(
        prepare_function=lambda data, is_test: pipe(data,[         
            lambda d: normalize_layers(d,not is_test),
            lambda d: np.expand_dims(d, axis=-1),
        ]),
        epochs=epochs,
        optimizer=optimizer
        )
)

params_crnn = CRNNNetworkParams(
    conv=[Conv3DParams(kernel=(1,10,10),pool_size=(1,10,10),filters=50,normalization=True,dropout=drop)],
    rec=[RecurrentLayerParams(type=tf.keras.layers.LSTM,units=125,bidirectional=True,dropout=drop),
         RecurrentLayerParams(type=tf.keras.layers.LSTM,units=125,bidirectional=True,dropout=drop)],
    flatten=FlattenParams(dropout=drop,normalization=True),
    deep=[DeepLayerParams(neurons=1000,dropout=drop,normalization=True),
          DeepLayerParams(neurons=400,dropout=drop,normalization=True)],
    generic=GenericNeuralNetworkParams(
        prepare_function=lambda data, is_test: pipe(data,[
            lambda d: split_3d_to_timeseries(d,15)
        ]),
        epochs=epochs,
        optimizer=optimizer
        )
)


params_svr = SVRParams(lambda data, is_test: normalize_layers(data,not is_test))
params_rf = RandomForestParams(lambda data, is_test: normalize_layers(data, not is_test))

BM3D

In [4]:
params_cnn2d_bm3d = CNN2DNetworkParams(
    conv=[Conv2DParams(kernel=(5,5),pool_size=(2,2),filters=64,normalization=True,activation="relu", dropout=drop),
          Conv2DParams(kernel=(3,3),pool_size=(2,2),filters=512, normalization=True,activation="relu", dropout=drop),
          Conv2DParams(kernel=(2,2),pool_size=(2,2),filters=512, normalization=True,activation="relu", dropout=drop),
          ],
    flatten=FlattenParams(dropout=drop,normalization=True),
    deep=[DeepLayerParams(neurons=1000,dropout=drop,normalization=True),
          DeepLayerParams(neurons=400,dropout=drop,normalization=True)],
    generic=GenericNeuralNetworkParams(
        prepare_function=lambda data, is_test: pipe(data,[lambda d: d]), 
        epochs=epochs,
        optimizer=optimizer
        )
)

params_crnn_bm3d = CRNNNetworkParams(
    conv=[Conv3DParams(kernel=(1,10,10),pool_size=(1,10,10),filters=50,normalization=True,dropout=drop)],
    rec=[RecurrentLayerParams(type=tf.keras.layers.LSTM,units=125,bidirectional=True,dropout=drop),
         RecurrentLayerParams(type=tf.keras.layers.LSTM,units=125,bidirectional=True,dropout=drop)],
    flatten=FlattenParams(dropout=drop,normalization=True),
    deep=[DeepLayerParams(neurons=1000,dropout=drop,normalization=True),
          DeepLayerParams(neurons=400,dropout=drop,normalization=True)],
    generic=GenericNeuralNetworkParams(
        prepare_function=lambda data, is_test: pipe(data,[
            lambda d: split_3d_to_timeseries(d,10)
        ]),
        epochs=epochs,
        optimizer=optimizer
        )
)

Spectral Indexes

In [5]:
params_svr_spec = SVRParams(lambda data, is_test: np.reshape(data,(data.shape[0],-1)))
params_rf_spec = RandomForestParams(lambda data, is_test: np.reshape(data,(data.shape[0],-1)))

Quantiles

In [6]:
params_cnn1d_quan = CNN1DNetworkParams(
    conv=[Conv1DParams(kernel=10,pool_size=5,filters=500,normalization=True,activation="relu", dropout=drop),
          Conv1DParams(kernel=5,pool_size=5,filters=1500, normalization=True,activation="relu", dropout=drop)
          ],
    deep=[DeepLayerParams(neurons=1000,dropout=drop,normalization=True),
          DeepLayerParams(neurons=400,dropout=drop,normalization=True)],
          flatten=FlattenParams(dropout=drop,normalization=True),
    generic=GenericNeuralNetworkParams(
        prepare_function=lambda data, is_test: data,
        epochs=5,
        optimizer=optimizer
        )
)

In [7]:
from ModelExperiment import ModelExperiment

experiment = ModelExperiment()
experiment.run_experiment(
    params_vector=[params_rf_spec],
    data=[all_data],
    gt=[gt["K"].values],
    folds=folds,
    test_data=[test_data]
)

In [8]:
experiment.models[0].save_history('./experiment2/cnn1d_dwt_pH.csv')
experiment.models[1].save_history('./experiment2/cnn2d_dwt_Mg.csv')

In [9]:
experiment.get_errors_report('./experiment2/errors_dwt_pH_cnn1d.csv')

In [8]:
print(experiment.models[0].crossval_scores)
print(experiment.models[0].crossval_rs)
print(experiment.models[0].mae)

[0.92615875 0.94365507 0.93417781 0.91160317 0.95003885]
[0.0734853  0.05497766 0.06574003 0.08832193 0.04731133]
[42.03165533 42.0205072  42.6650948  43.12611214 44.9043815 ]


In [15]:
print(experiment.models[1].crossval_scores)
print(experiment.models[1].crossval_rs)
print(experiment.models[1].mae)

[1.03476658 1.06703741 0.93329347 1.01009731 1.01159021]
[-0.03562846 -0.06755331  0.06670624 -0.01012128 -0.01159023]
[20.23006225 20.25107089 20.03343006 21.22519769 21.74225549]
